# Hybrid CNN + Quantum Neural Network for CIFAR-10
Implements a density QNN architecture based on the paper's framework

In [11]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import pennylane as qml
import numpy as np

## Environment Setup
Load AWS configuration from .env file

In [12]:
import os
from dotenv import load_dotenv
from IPython.display import clear_output

# Load environment variables from .env file
load_dotenv()

# Function to safely get and mask sensitive environment variables
def get_masked_env(var_name):
    value = os.getenv(var_name, '')
    if value and var_name in ['AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY']:
        return value[:4] + '*' * (len(value) - 8) + value[-4:]
    return value

# Display current configuration
print("Current AWS Configuration:")
print(f"Region: {get_masked_env('AWS_DEFAULT_REGION')}")
print(f"Access Key ID: {get_masked_env('AWS_ACCESS_KEY_ID')}")
print(f"Secret Access Key: {get_masked_env('AWS_SECRET_ACCESS_KEY')}")
print(f"Braket Device: {get_masked_env('BRAKET_DEVICE')}")

# Function to update AWS configuration
def update_aws_config(region=None, access_key=None, secret_key=None, device_arn=None):
    if region:
        os.environ['AWS_DEFAULT_REGION'] = region
    if access_key:
        os.environ['AWS_ACCESS_KEY_ID'] = access_key
    if secret_key:
        os.environ['AWS_SECRET_ACCESS_KEY'] = secret_key
    if device_arn:
        os.environ['BRAKET_DEVICE'] = device_arn
    
    clear_output()
    print("Updated AWS Configuration:")
    print(f"Region: {get_masked_env('AWS_DEFAULT_REGION')}")
    print(f"Access Key ID: {get_masked_env('AWS_ACCESS_KEY_ID')}")
    print(f"Secret Access Key: {get_masked_env('AWS_SECRET_ACCESS_KEY')}")
    print(f"Braket Device: {get_masked_env('BRAKET_DEVICE')}")


Current AWS Configuration:
Region: us-east-1
Access Key ID: AKIA************ZMTY
Secret Access Key: RXLx********************************i0yI
Braket Device: arn:aws:braket:::device/quantum-simulator/amazon/sv1


In [13]:
# Load CIFAR-10 sample
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)
images, labels = next(iter(trainloader))

print(f"Loaded batch: {images.shape}")

Files already downloaded and verified
Loaded batch: torch.Size([4, 3, 32, 32])
Loaded batch: torch.Size([4, 3, 32, 32])


In [16]:
# Amazon Braket SV1 simulator device
dev = qml.device(
    "braket.aws.qubit",
    device_arn=os.getenv('BRAKET_DEVICE'),
    wires=4,
    shots=100  # Set number of shots here
)

In [17]:
# Density QNN: Sub-unitary quantum circuit (RBS-based)
@qml.qnode(dev, interface="torch")
def quantum_sub_circuit(inputs, weights):
    # Data encoding
    for i in range(4):
        qml.RY(inputs[i], wires=i)
    
    # Parameterized RBS-inspired gates
    for i in range(4):
        qml.RZ(weights[i], wires=i)
    
    # Entanglement (CNOT ladder)
    for i in range(3):
        qml.CNOT(wires=[i, i+1])
    
    # Second rotation layer
    for i in range(4):
        qml.RY(weights[i+4], wires=i)
    
    return [qml.expval(qml.PauliZ(i)) for i in range(4)]

In [22]:
# Hybrid CNN + Density Quantum Model
class HybridDensityQNN(nn.Module):
    def __init__(self, num_sub_unitaries=2):
        super(HybridDensityQNN, self).__init__()
        
        # CNN feature extractor
        self.conv1 = nn.Conv2d(3, 8, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 4)
        
        # Density QNN: K sub-unitaries with independent parameters
        self.K = num_sub_unitaries
        self.quantum_layers = nn.ModuleList([
            qml.qnn.TorchLayer(quantum_sub_circuit, {"weights": (8,)})
            for _ in range(self.K)
        ])
        
        # Trainable mixing coefficients α_k (ensure sum to 1 via softmax)
        self.alpha = nn.Parameter(torch.ones(self.K))
        
        # Final classifier
        self.fc2 = nn.Linear(4, 10)
    
    def forward(self, x):
        # CNN feature extraction
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = torch.tanh(self.fc1(x))
        
        # Density QNN: weighted sum of sub-unitaries
        alpha_norm = torch.softmax(self.alpha, dim=0)
        quantum_out = sum(alpha_norm[k] * self.quantum_layers[k](x) for k in range(self.K))
        
        # Classification
        out = self.fc2(quantum_out)
        return out

In [23]:
# Initialize and test forward pass
model = HybridDensityQNN(num_sub_unitaries=2)
output = model(images)

print(f"Input shape: {images.shape}")
print(f"Output shape: {output.shape}")
print(f"Mixing coefficients α: {torch.softmax(model.alpha, dim=0).detach()}")
print(f"\nOutput logits:\n{output}")

Input shape: torch.Size([4, 3, 32, 32])
Output shape: torch.Size([4, 10])
Mixing coefficients α: tensor([0.5000, 0.5000])

Output logits:
tensor([[ 0.2526, -0.3160,  0.0282,  0.6928,  0.0157, -0.7796, -0.5577,  0.0707,
          0.1227, -0.4648],
        [ 0.2553, -0.3378,  0.0234,  0.6888,  0.0476, -0.7318, -0.5319,  0.0629,
          0.0822, -0.4633],
        [ 0.2495, -0.3601,  0.0563,  0.6795, -0.0025, -0.7777, -0.5845,  0.0661,
          0.1195, -0.4718],
        [ 0.2814, -0.3382,  0.0272,  0.7245,  0.0389, -0.7296, -0.5493,  0.0232,
          0.0646, -0.4749]], grad_fn=<AddmmBackward0>)


## Output Analysis

Let's analyze the model's output and performance:

In [40]:
# Create Fair Comparison Models
print("\n" + "="*60)
print("FAIR MODEL COMPARISON: CNN vs Hybrid QNN")
print("="*60)

# Define pure CNN baseline (same capacity as hybrid)
class PureCNN(nn.Module):
    def __init__(self):
        super(PureCNN, self).__init__()
        # Same CNN feature extractor
        self.conv1 = nn.Conv2d(3, 8, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 4)
        
        # Extra classical layer to match quantum layer capacity
        # Quantum has 2 sub-unitaries * 8 params each = 16 params
        # Plus 4->10 final layer
        self.fc_quantum_equiv = nn.Linear(4, 4)  # Replaces quantum layer
        
        # Final classifier
        self.fc2 = nn.Linear(4, 10)
    
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = torch.tanh(self.fc1(x))
        x = torch.relu(self.fc_quantum_equiv(x))  # Classical "quantum" layer
        x = self.fc2(x)
        return x

# Initialize both models
pure_cnn = PureCNN()
hybrid_qnn = model  # Use existing model

# Count parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

cnn_params = count_parameters(pure_cnn)
hybrid_params = count_parameters(hybrid_qnn)

print("\nModel Architecture Comparison:")
print("-"*60)
print(f"Pure CNN Parameters:        {cnn_params:,}")
print(f"Hybrid QNN Parameters:      {hybrid_params:,}")
print(f"Parameter Difference:       {abs(hybrid_params - cnn_params):,}")

# Run inference on same batch
with torch.no_grad():
    cnn_output = pure_cnn(images)
    hybrid_output = hybrid_qnn(images)
    
    cnn_predictions = torch.argmax(cnn_output, dim=1)
    hybrid_predictions = torch.argmax(hybrid_output, dim=1)
    
    cnn_probs = torch.softmax(cnn_output, dim=1)
    hybrid_probs = torch.softmax(hybrid_output, dim=1)

# CIFAR-10 classes
classes = ('plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')

# Convert labels
if isinstance(labels, list):
    labels_tensor = torch.tensor([classes.index(label) for label in labels])
else:
    labels_tensor = labels

# Display predictions
print("\n" + "-"*60)
print("PREDICTION COMPARISON (Untrained Models)")
print("-"*60)
print(f"{'Image':<8} {'True':<10} {'Pure CNN':<12} {'Hybrid QNN':<12} {'Match?':<8}")
print("-"*60)

matches = 0
for i in range(len(images)):
    true_label = classes[labels_tensor[i]]
    cnn_pred = classes[cnn_predictions[i]]
    hybrid_pred = classes[hybrid_predictions[i]]
    match = "✓" if cnn_pred == hybrid_pred else "✗"
    
    if cnn_pred == hybrid_pred:
        matches += 1
    
    print(f"{i+1:<8} {true_label:<10} {cnn_pred:<12} {hybrid_pred:<12} {match:<8}")

print("-"*60)
print(f"Prediction Agreement: {matches}/{len(images)} ({matches/len(images)*100:.0f}%)")

# Confidence comparison
print("\n" + "-"*60)
print("CONFIDENCE SCORES")
print("-"*60)
print(f"{'Image':<8} {'CNN Conf':<15} {'Hybrid Conf':<15} {'Difference':<12}")
print("-"*60)

for i in range(len(images)):
    cnn_conf = cnn_probs[i][cnn_predictions[i]] * 100
    hybrid_conf = hybrid_probs[i][hybrid_predictions[i]] * 100
    diff = hybrid_conf - cnn_conf
    
    print(f"{i+1:<8} {cnn_conf:>6.2f}%{'':<8} {hybrid_conf:>6.2f}%{'':<8} {diff:>+6.2f}%")

print("-"*60)
print(f"Avg CNN Confidence:    {cnn_probs.max(dim=1)[0].mean()*100:.2f}%")
print(f"Avg Hybrid Confidence: {hybrid_probs.max(dim=1)[0].mean()*100:.2f}%")

# Feature space analysis
print("\n" + "-"*60)
print("FEATURE SPACE ANALYSIS")
print("-"*60)

with torch.no_grad():
    # CNN intermediate features
    x = pure_cnn.pool(torch.relu(pure_cnn.conv1(images)))
    x = pure_cnn.pool(torch.relu(pure_cnn.conv2(x)))
    x = x.view(-1, 16 * 5 * 5)
    cnn_pre_features = pure_cnn.fc1(x)
    cnn_features = torch.relu(pure_cnn.fc_quantum_equiv(torch.tanh(cnn_pre_features)))
    
    # Hybrid quantum features
    x = hybrid_qnn.pool(torch.relu(hybrid_qnn.conv1(images)))
    x = hybrid_qnn.pool(torch.relu(hybrid_qnn.conv2(x)))
    x = x.view(-1, 16 * 5 * 5)
    hybrid_pre_features = hybrid_qnn.fc1(x)
    
    quantum_features = torch.tanh(hybrid_pre_features)
    alpha_norm = torch.softmax(hybrid_qnn.alpha, dim=0)
    
    quantum_outputs = []
    for sample in quantum_features:
        circuit_outs = [layer(sample) for layer in hybrid_qnn.quantum_layers]
        weighted_out = sum(alpha_norm[k] * circuit_outs[k] for k in range(hybrid_qnn.K))
        quantum_outputs.append(weighted_out)
    quantum_outputs = torch.stack(quantum_outputs)

print("\nPure CNN Feature Layer:")
print(f"  Range: [{cnn_features.min():.3f}, {cnn_features.max():.3f}]")
print(f"  Mean:  {cnn_features.mean():.3f} ± {cnn_features.std():.3f}")
print(f"  Sparsity: {(cnn_features == 0).float().mean()*100:.1f}% zeros")

print("\nHybrid Quantum Layer:")
print(f"  Range: [{quantum_outputs.min():.3f}, {quantum_outputs.max():.3f}]")
print(f"  Mean:  {quantum_outputs.mean():.3f} ± {quantum_outputs.std():.3f}")
print(f"  Mixing α: {alpha_norm.detach().numpy()}")

print("\n" + "="*60)
print("NOTE: Models are untrained - performance will improve after training")
print("="*60)


FAIR MODEL COMPARISON: CNN vs Hybrid QNN

Model Architecture Comparison:
------------------------------------------------------------
Pure CNN Parameters:        5,498
Hybrid QNN Parameters:      5,496
Parameter Difference:       2

------------------------------------------------------------
PREDICTION COMPARISON (Untrained Models)
------------------------------------------------------------
Image    True       Pure CNN     Hybrid QNN   Match?  
------------------------------------------------------------
1        plane      plane        cat          ✗       
2        horse      plane        cat          ✗       
3        dog        plane        cat          ✗       
4        horse      plane        cat          ✗       
------------------------------------------------------------
Prediction Agreement: 0/4 (0%)

------------------------------------------------------------
CONFIDENCE SCORES
------------------------------------------------------------
Image    CNN Conf        Hybrid Co

In [39]:
# Simplified Performance Comparison
print("\n" + "="*60)
print("MODEL PERFORMANCE COMPARISON")
print("="*60)

# 1. Test CNN-only path (without quantum layer)
with torch.no_grad():
    # Extract CNN features
    x = model.pool(torch.relu(model.conv1(images)))
    x = model.pool(torch.relu(model.conv2(x)))
    x = x.view(-1, 16 * 5 * 5)
    cnn_features = model.fc1(x)
    
    # CNN-only prediction (skip quantum, go straight to classifier)
    cnn_only_output = model.fc2(cnn_features)
    cnn_predictions = torch.argmax(cnn_only_output, dim=1)
    
    # Full hybrid model prediction
    hybrid_output = model(images)
    hybrid_predictions = torch.argmax(hybrid_output, dim=1)

# CIFAR-10 classes
classes = ('plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')

# Convert labels to tensor if needed
if isinstance(labels, list):
    labels_tensor = torch.tensor([classes.index(label) for label in labels])
else:
    labels_tensor = labels

# Display results
print("\n" + "-"*60)
print("BATCH PREDICTIONS (4 images)")
print("-"*60)
print(f"{'Image':<8} {'True Label':<12} {'CNN Only':<12} {'Hybrid QNN':<12}")
print("-"*60)

for i in range(len(images)):
    true_label = classes[labels_tensor[i]]
    cnn_pred = classes[cnn_predictions[i]]
    hybrid_pred = classes[hybrid_predictions[i]]
    
    print(f"{i+1:<8} {true_label:<12} {cnn_pred:<12} {hybrid_pred:<12}")

# Compare confidence scores
print("\n" + "-"*60)
print("CONFIDENCE SCORES (Top prediction)")
print("-"*60)

cnn_probs = torch.softmax(cnn_only_output, dim=1)
hybrid_probs = torch.softmax(hybrid_output, dim=1)

for i in range(len(images)):
    cnn_conf = cnn_probs[i][cnn_predictions[i]] * 100
    hybrid_conf = hybrid_probs[i][hybrid_predictions[i]] * 100
    
    print(f"Image {i+1}: CNN={cnn_conf:.1f}%  |  Hybrid={hybrid_conf:.1f}%")

# Feature space statistics
print("\n" + "-"*60)
print("FEATURE SPACE ANALYSIS")
print("-"*60)

# Get quantum features
with torch.no_grad():
    quantum_features = torch.tanh(cnn_features)
    alpha_norm = torch.softmax(model.alpha, dim=0)
    
    # Process each sample individually
    quantum_outputs = []
    for sample in quantum_features:
        circuit_outs = [layer(sample) for layer in model.quantum_layers]
        weighted_out = sum(alpha_norm[k] * circuit_outs[k] for k in range(model.K))
        quantum_outputs.append(weighted_out)
    quantum_outputs = torch.stack(quantum_outputs)

print(f"CNN Feature Statistics:")
print(f"  Range: [{cnn_features.min():.3f}, {cnn_features.max():.3f}]")
print(f"  Mean: {cnn_features.mean():.3f}, Std: {cnn_features.std():.3f}")

print(f"\nQuantum Feature Statistics:")
print(f"  Range: [{quantum_outputs.min():.3f}, {quantum_outputs.max():.3f}]")
print(f"  Mean: {quantum_outputs.mean():.3f}, Std: {quantum_outputs.std():.3f}")

print(f"\nQuantum Layer Info:")
print(f"  Sub-unitaries: {model.K}")
print(f"  Mixing coefficients α: {alpha_norm.detach().numpy()}")
print(f"  Qubit count: 4")
print(f"  Parameters per circuit: 8")

print("\n" + "="*60)


MODEL PERFORMANCE COMPARISON

------------------------------------------------------------
BATCH PREDICTIONS (4 images)
------------------------------------------------------------
Image    True Label   CNN Only     Hybrid QNN  
------------------------------------------------------------
1        plane        deer         cat         
2        horse        deer         cat         
3        dog          deer         cat         
4        horse        deer         cat         

------------------------------------------------------------
CONFIDENCE SCORES (Top prediction)
------------------------------------------------------------
Image 1: CNN=14.9%  |  Hybrid=20.1%
Image 2: CNN=15.4%  |  Hybrid=20.4%
Image 3: CNN=14.5%  |  Hybrid=20.1%
Image 4: CNN=15.6%  |  Hybrid=20.7%

------------------------------------------------------------
FEATURE SPACE ANALYSIS
------------------------------------------------------------

------------------------------------------------------------
BATCH P